<a href="https://colab.research.google.com/github/sarthakc123/Sarthak_LLM_Research/blob/main/Sarthak_Research_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install chromadb sentence-transformers
!pip install -q transformers accelerate

import time, re
from sentence_transformers import SentenceTransformer
import chromadb
#from llama_cpp import Llama
import matplotlib.pyplot as plt

In [8]:
from google.colab import drive
import json
import os

drive.mount('/content/drive')
log_path_json = "/content/drive/MyDrive/conversation_log.json"
log_path_txt = "/content/drive/MyDrive/conversation_log.txt"

# Load existing log or start fresh
if os.path.exists(log_path_json):
    with open(log_path_json, "r") as f:
        conversation_log = json.load(f)
    print("✅ Loaded existing conversation log.")
else:
    conversation_log = []
    print("🆕 Starting new conversation log.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🆕 Starting new conversation log.


In [ ]:
!mkdir -p /content/models
!wget -O /content/models/mistral-7b-instruct-v0.2.Q4_K_M.gguf \
https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf

--2025-05-05 17:11:26--  https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.55, 18.164.174.17, 18.164.174.23, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/72/62/726219e98582d16c24a66629a4dec1b0761b91c918e15dea2625b4293c134a92/3e0039fd0273fcbebb49228943b17831aadd55cbcbf56f0af00499be2040ccf9?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27mistral-7b-instruct-v0.2.Q4_K_M.gguf%3B+filename%3D%22mistral-7b-instruct-v0.2.Q4_K_M.gguf%22%3B&Expires=1746468686&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NjQ2ODY4Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzcyLzYyLzcyNjIxOWU5ODU4MmQxNmMyNGE2NjYyOWE0ZGVjMWIwNzYxYjkxYzkxOGUxNWRlYTI2MjViNDI5M2MxMzRhOTIvM2UwMDM5ZmQwMjczZmNiZWJi

In [4]:
from huggingface_hub import login
login()

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",  # will use GPU if available
    torch_dtype=torch.float16,
    trust_remote_code=True
)

def llama_completion(prompt: str, max_tokens=1024):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_tokens, do_sample=True, temperature=0.7, top_p=0.95)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).replace(prompt, "").strip()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [6]:
import chromadb
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection("conversation_memory")

def embed_fn(texts): return embedding_model.encode(texts).tolist()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
short_term_buffer = []

def truncate_text(text, max_tokens=1500):
    words = text.split()
    return " ".join(words[-max_tokens:]) if len(words) > max_tokens else text

def add_to_vector_memory(agent_name, message):
    collection.add(
        documents=[message],
        metadatas=[{"agent": agent_name}],
        ids=[f"{agent_name}_{str(time.time())}"],
        embeddings=embed_fn([message])
    )

def get_context(agent_name, k=5):
    # Use last few messages to form a meaningful query
    query_text = "\n".join(short_term_buffer[-3:]) if short_term_buffer else "AI-powered fitness app for seniors"

    results = collection.query(query_embeddings=embed_fn([query_text]), n_results=k)

    docs = results['documents']
    semantic_context = "\n".join(sum(docs, [])) if docs else ""

    # Working memory from other agents
    working = "\n".join([
        line for line in short_term_buffer[-5:]
        if not line.startswith(f"{agent_name}:")
    ])

    return truncate_text(working + "\n\n" + semantic_context, max_tokens=1500)

In [10]:
class AgentWithMemory:
    def __init__(self, name, system_message, window=5):
        self.name = name
        self.system_message = system_message
        self.window = window

    def generate_reply(self, extra_prompt=""):
        context = get_context(self.name, self.window)
        novelty_prompt = "Ensure your response introduces new ideas and avoids repeating previously discussed concepts."
        full_prompt = f"{self.system_message}\n\n{novelty_prompt}\n{extra_prompt}\n\nContext:\n{context}\n\n{self.name}, continue:"
        reply = llama_completion(full_prompt)
        add_to_vector_memory(self.name, reply)
        short_term_buffer.append(f"{self.name}:\n{reply}")
        return {"name": self.name, "content": reply}

class MetaCritic:
    def __init__(self):
        self.name = "MetaCritic"

    def critique_and_modify_prompt(self, round_text):
        prompt = f"""You are MetaCritic, an evaluator AI. Analyze the following agent discussion and help guide the next round.

Round Transcript:
{truncate_text(round_text, max_tokens=1200)}

Evaluate:
- Are the agents repeating concepts?
- Are new insights being added?
- Are the Agents building up on the conversation?
- Is the discussion actually of value? Give a score out of 1.
- Suggest how agents should revise their approach next round.

Output:
1. Summary of issues
2. A motivational, precise instruction to guide the next round.
"""
        response = llama_completion(prompt)
        add_to_vector_memory(self.name, response)
        short_term_buffer.append(f"{self.name}:\n{response}")
        return response

In [11]:
project_manager = AgentWithMemory("ProjectManager", "You are a Project Manager. Define milestones and plan the execution of the project.")
tech_lead = AgentWithMemory("TechnicalLead", "You are a Technical Lead. Propose architecture, research tech stack and create a solution architecture.")
biz_analyst = AgentWithMemory("BusinessAnalyst", "You are a Business Analyst. Set KPIs. Create a cost benefit analysis too see if the project can make a good business impact.")
critic = MetaCritic()

agents = [project_manager, tech_lead, biz_analyst]
round_logs = []
conversation_log = []

initial_message = "We want to build an AI-powered fitness app for seniors. Study the market, cresearch, ollaborate and draft a complete proposal."
add_to_vector_memory("User", initial_message)
short_term_buffer.append("User:\n" + initial_message)


In [12]:
prompt_modifier = ""
start_round = len(conversation_log)  # Resume if file exists

for i in range(start_round, start_round + 5):
    print(f"\n🔁 Round {i+1}")
    round_data = {
        "round": i + 1,
        "prompt_modifier": prompt_modifier,
        "agent_responses": [],
        "metacritic_feedback": ""
    }

    round_text = ""
    for agent in agents:
        reply = agent.generate_reply(extra_prompt=prompt_modifier)
        print(f"{reply['name']}:\n{reply['content']}\n")
        round_data["agent_responses"].append(reply)
        round_text += f"{reply['name']}:\n{reply['content']}\n\n"

    feedback = critic.critique_and_modify_prompt(round_text)
    print(f"\n🧠 MetaCritic:\n{feedback}")
    round_data["metacritic_feedback"] = feedback
    prompt_modifier = feedback

    # Save round
    conversation_log.append(round_data)
    with open(log_path_json, "w") as f:
        json.dump(conversation_log, f, indent=2)

    # Also save as readable .txt
    with open(log_path_txt, "w") as f:
        for round_data in conversation_log:
            f.write(f"\n🔁 Round {round_data['round']}\n")
            for reply in round_data["agent_responses"]:
                f.write(f"{reply['name']}:\n{reply['content']}\n\n")
            f.write(f"🧠 MetaCritic:\n{round_data['metacritic_feedback']}\n\n")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🔁 Round 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ProjectManager:
First and foremost, we'll conduct thorough market research to identify our competition, target audience, and market size. This stage includes:

1. Market Analysis: Identify competitors, market size, and trends.
2. Target Audience Research: Understand the needs and preferences of seniors.
3. SWOT Analysis: Evaluate Strengths, Weaknesses, Opportunities, and Threats.

Next, we'll begin the design and development phase, which can be broken down into the following milestones:

1. Information Architecture: Create the app's structure, including user flows and wireframes.
2. User Interface Design: Develop a visually appealing and user-friendly design.
3. User Experience Design: Ensure the app is easy to navigate and engaging.
4. Front-end Development: Implement the design using HTML, CSS, and JavaScript.
5. Back-end Development: Develop the server-side functionality and database.
6. Integration: Combine the front-end and back-end to create a complete app.
7. Testing: Perform ri

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

ProjectManager:
We've identified a number of features and technologies for our AI-powered fitness app for seniors. In the next phase of the project, we'll focus on refining the user experience and user interface design to ensure that our app is not only functional but also engaging and delightful for seniors. To do this, we'll conduct user research and usability testing to identify pain points and areas for improvement. We'll also explore design patterns and best practices for creating intuitive and accessible user interfaces for seniors.

UserResearcher, what user research methods would you recommend for this project?

UserResearcher: I would recommend a combination of qualitative and quantitative research methods for this project. Here are some methods we could consider:

1. Interviews: Conduct in-depth interviews with seniors to understand their fitness goals, motivations, pain points, and preferences. This will help us gain a deep understanding of their unique needs and expectation

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


TechnicalLead:
Based on the discussion so far, I suggest we adopt a hybrid mobile application development approach using React Native for the frontend and Node.js for the backend. This stack offers several advantages, including cross-platform compatibility, a large developer community, and the ability to build both the frontend and backend using JavaScript.

For the fall detection system, we can leverage machine learning algorithms, specifically, we can use TensorFlow Lite to run machine learning models locally on the user's device. This approach ensures user privacy and reduces latency as the model can process data in real-time without requiring an internet connection.

To provide personalized fitness recommendations, we can implement a recommendation engine using collaborative filtering and content-based filtering techniques. We can also consider using natural language processing (NLP) to analyze user feedback and identify patterns to further refine recommendations.

For real-time an

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


BusinessAnalyst:
To ensure the success of our AI-powered fitness app for seniors, we must establish clear Key Performance Indicators (KPIs) that align with the project's objectives and goals. These KPIs will provide us with valuable insights into the app's performance and effectiveness, enabling us to make data-driven decisions and optimize the user experience.

Here are some potential KPIs for our AI-powered fitness app for seniors:

1. User Engagement: This KPI measures the level of user interaction and usage of the app. It can include metrics such as daily active users (DAU), monthly active users (MAU), average session length, and user retention rate.
2. User Satisfaction: This KPI measures the overall satisfaction of our users with the app. It can be assessed through surveys, user feedback, and app store ratings.
3. Revenue: This KPI measures the financial performance of the app, including subscription fees, in-app purchases, and advertising revenue.
4. Retention: This KPI measures

In [18]:
class ActRAgent:
    def __init__(self, name="ActRAgent", memory_top_k=10):
        self.name = name
        self.memory_top_k = memory_top_k

    def make_final_decision(self, topic="AI-powered fitness app for seniors"):
        # Retrieve top relevant memory chunks from vector DB
        results = collection.query(
            query_embeddings=embed_fn([topic]),
            n_results=self.memory_top_k
        )
        semantic_memory = "\n".join(sum(results['documents'], [])) if results['documents'] else ""

        # Short-term working memory (last few exchanges)
        working_memory = "\n".join(short_term_buffer[-5:]) if short_term_buffer else ""

        # Combine both as ACT-R style memory context
        memory_context = truncate_text(working_memory + "\n\n" + semantic_memory, max_tokens=1600)

        prompt = f"""
You are an expert cognitive agent inspired by the ACT-R architecture.

Your task is to analyze this conversation among LLM agents collaborating on an AI-powered fitness app for seniors.
Based on your cognitive memory (retrieved semantically), decide if the project is ready to move forward.

Evaluate on:
- Novelty and insight
- Technical completeness
- Business value and KPIs
- Cost & ROI
- Reduced redundancy
- Evidence of agent collaboration

Conversation Memory Context:
{memory_context}

Respond with:
1. Final Decision: (Proceed / Rework / Reject)
2. Rationale: A short paragraph with reasoning.
3. Give a confidence score out of 1

Answer:
"""
        response = llama_completion(prompt)
        short_term_buffer.append(f"{self.name}:\n{response}")
        add_to_vector_memory(self.name, response)
        return response

In [19]:
# Instantiate ACT-R agent and get final decision
actr = ActRAgent()
final_decision = actr.make_final_decision(topic="AI-powered fitness app for seniors")

print("\n📌 FINAL DECISION by ACT-R Agent:\n")
print(final_decision)

# Optional save to file
with open("/content/drive/MyDrive/final_actr_decision.txt", "w") as f:
    f.write(final_decision)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



📌 FINAL DECISION by ACT-R Agent:

1. Proceed
2. The conversation among the agents shows a clear understanding of the unique needs and preferences of their target audience, seniors, and a well-thought-out plan for features, architecture, and tech stack to meet those needs. They also discuss the importance of user research and usability testing, which is crucial for creating an exceptional user experience. Therefore, based on the information provided, I believe the project is ready to move forward.
3. 1 (I have high confidence in the project's readiness to move forward)


In [32]:
def validate_rq1_actr_coherence(final_decision_text, memory_snippets):
    import nltk
    from nltk.translate.bleu_score import sentence_bleu
    nltk.download("punkt")

    # Check that memory items are semantically referenced
    ref_score = sum(1 for chunk in memory_snippets if chunk.lower()[:20] in final_decision_text.lower())
    coherence = len(final_decision_text.split()) > 80  # Long-form response
    rationale_present = "rationale" in final_decision_text.lower()

    return {
        "uses_memory": ref_score >= 2,
        "has_coherence": coherence,
        "has_rationale": rationale_present
    }

In [38]:
def validate_rq2_metacritic_evolution(conversation_log):
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity

    feedbacks = [round["metacritic_feedback"] for round in conversation_log]
    lengths = [len(f.split()) for f in feedbacks]
    lexical_diversity = len(set(" ".join(feedbacks).split())) / sum(lengths)

    vectorizer = TfidfVectorizer()
    vecs = vectorizer.fit_transform(feedbacks)

    num_vecs = vecs.shape[0]
    avg_cos_sim = sum(
        cosine_similarity(vecs[i], vecs[i + 1])[0][0]
        for i in range(num_vecs - 1)
    ) / (num_vecs - 1)

    return {
        "lexical_diversity": round(lexical_diversity, 2),
        "avg_feedback_similarity": round(avg_cos_sim, 2),
        "improving_feedback": lengths[-1] > lengths[0]
    }

In [40]:
def validate_rq3_novelty(conversation_log):
    import re
    keyword_sets = []
    jaccard_scores = []

    for round_data in conversation_log:
        text = " ".join([a["content"] for a in round_data["agent_responses"]])
        keywords = set(re.findall(r'\b[a-zA-Z]{5,}\b', text.lower()))
        keyword_sets.append(keywords)

    for i in range(1, len(keyword_sets)):
        inter = keyword_sets[i] & keyword_sets[i-1]
        union = keyword_sets[i] | keyword_sets[i-1]
        jaccard_scores.append(len(inter) / len(union) if union else 0)

    novelty_per_round = [len(keyword_sets[i] - set.union(set(), *keyword_sets[:i])) for i in range(len(keyword_sets))]

    return {
        "avg_jaccard_redundancy": round(sum(jaccard_scores) / len(jaccard_scores), 2),
        "avg_new_keywords": round(sum(novelty_per_round) / len(novelty_per_round), 2),
        "min_new_keywords": min(novelty_per_round)
    }


In [35]:
def validate_rq4_convergence(conversation_log):
    collab_terms = ["build on", "integrate", "cohesive", "responding to", "reduce repetition"]
    count = sum(
        sum(term in r["metacritic_feedback"].lower() for term in collab_terms)
        for r in conversation_log
    )
    return {
        "collaboration_suggestions": count,
        "mentions_per_round": round(count / len(conversation_log), 2)
    }

In [36]:
def validate_rq5_emergence(conversation_log):
    agent_refs = 0
    for round_data in conversation_log:
        for reply in round_data["agent_responses"]:
            agent_refs += sum(ref in reply["content"].lower() for ref in ["projectmanager", "technicallead", "businessanalyst"])

    return {
        "agent_cross_references": agent_refs,
        "average_refs_per_round": round(agent_refs / len(conversation_log), 2)
    }

In [41]:
print("🧠 ADVANCED RESEARCH VALIDATION REPORT\n")

# Pull memory results if needed for RQ1
vector_results = collection.query(
    query_embeddings=embed_fn(["AI-powered fitness app for seniors"]),
    n_results=5
)
memory_snippets = sum(vector_results['documents'], [])

report = {
    "RQ1_ACTR": validate_rq1_actr_coherence(final_decision, memory_snippets),
    "RQ2_MetaCritic": validate_rq2_metacritic_evolution(conversation_log),
    "RQ3_Novelty": validate_rq3_novelty(conversation_log),
    "RQ4_Convergence": validate_rq4_convergence(conversation_log),
    "RQ5_Emergence": validate_rq5_emergence(conversation_log),
}

from pprint import pprint
pprint(report)

🧠 ADVANCED RESEARCH VALIDATION REPORT

{'RQ1_ACTR': {'has_coherence': True,
              'has_rationale': False,
              'uses_memory': False},
 'RQ2_MetaCritic': {'avg_feedback_similarity': np.float64(0.67),
                    'improving_feedback': True,
                    'lexical_diversity': 0.28},
 'RQ3_Novelty': {'avg_jaccard_redundancy': 0.33,
                 'avg_new_keywords': 170.8,
                 'min_new_keywords': 62},
 'RQ4_Convergence': {'collaboration_suggestions': 2, 'mentions_per_round': 0.4},
 'RQ5_Emergence': {'agent_cross_references': 6, 'average_refs_per_round': 1.2}}


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [42]:
report = {
    "RQ1_ACTR": validate_rq1_actr_coherence(final_decision, memory_snippets),
    "RQ2_MetaCritic": validate_rq2_metacritic_evolution(conversation_log),
    "RQ3_Novelty": validate_rq3_novelty(conversation_log),
    "RQ4_Convergence": validate_rq4_convergence(conversation_log),
    "RQ5_Emergence": validate_rq5_emergence(conversation_log),
}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
